In [78]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import glob
from collections import defaultdict
from sklearn.metrics import confusion_matrix
import seaborn as sns

def load_predictions(predictions_file):
    """Load YOLO predictions from JSON file."""
    with open(predictions_file, 'r') as f:
        predictions = json.load(f)
    
    # Group predictions by image_id
    pred_by_image = defaultdict(list)
    for pred in predictions:
        image_id = pred["image_id"]
        bbox = pred["bbox"]  # [x, y, width, height] format
        score = pred["score"]
        category_id = pred["category_id"]
        
        pred_by_image[image_id].append({
            "bbox": bbox,
            "score": score,
            "category_id": category_id
        })
    
    return pred_by_image

def load_ground_truth(labels_dir):
    """Load ground truth YOLO labels from directory."""
    gt_by_image = {}
    
    # Get all .txt files in the labels directory
    label_files = glob.glob(os.path.join(labels_dir, "**/*.txt"), recursive=True)
    
    for label_file in label_files:
        # Extract image_id from filename
        filename = os.path.basename(label_file)
        image_id = os.path.splitext(filename)[0]
        
        boxes = []
        # YOLO format: class_id x_center y_center width height (normalized)
        with open(label_file, 'r') as f:
            for line in f:
                if line.strip():
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        x_center = float(parts[1])
                        y_center = float(parts[2])
                        width = float(parts[3])
                        height = float(parts[4])
                        
                        boxes.append({
                            "class_id": class_id,
                            "bbox_normalized": [x_center, y_center, width, height]
                        })
        
        gt_by_image[image_id] = boxes
    
    return gt_by_image

def convert_yolo_to_absolute(bbox_normalized, image_width=1920, image_height=1920):
    """Convert YOLO normalized bbox to absolute coordinates [x, y, w, h]"""
    x_center, y_center, width, height = bbox_normalized
    
    x = (x_center - width/2) * image_width
    y = (y_center - height/2) * image_height
    w = width * image_width
    h = height * image_height
    
    return [x, y, w, h]

def calculate_iou(box1, box2):
    """
    Calculate IoU between two bounding boxes
    Both boxes in format [x, y, w, h] (absolute coordinates)
    """
    # Calculate coordinates of both boxes
    x1_1, y1_1 = box1[0], box1[1]
    x2_1, y2_1 = box1[0] + box1[2], box1[1] + box1[3]
    
    x1_2, y1_2 = box2[0], box2[1]
    x2_2, y2_2 = box2[0] + box2[2], box2[1] + box2[3]
    
    # Calculate intersection area
    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)
    
    if x2_i < x1_i or y2_i < y1_i:
        return 0.0  # No intersection
        
    intersection_area = (x2_i - x1_i) * (y2_i - y1_i)
    
    # Calculate union area
    box1_area = box1[2] * box1[3]
    box2_area = box2[2] * box2[3]
    union_area = box1_area + box2_area - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area if union_area > 0 else 0
    
    return iou

def evaluate_predictions(predictions, ground_truth, iou_threshold=0.5, conf_threshold=0.01, image_width=1920, image_height=1920):
    """
    Evaluate predictions against ground truth
    Returns true and predicted labels for confusion matrix
    """
    y_true = []  # Ground truth labels 
    y_pred = []  # Predicted labels
    
    # Track metrics
    tp = 0  # True positives
    fp = 0  # False positives
    fn = 0  # False negatives
    
    for image_id, gt_boxes in ground_truth.items():
        # Get predictions for this image
        image_preds = predictions.get(image_id, [])
        
        # Filter by confidence threshold
        image_preds = [p for p in image_preds if p["score"] >= conf_threshold]
        
        # Initialize tracking arrays for this image
        gt_matched = [False] * len(gt_boxes)
        pred_matched = [False] * len(image_preds)
        
        # For each prediction, find best matching ground truth box
        for pred_idx, pred in enumerate(image_preds):
            pred_bbox = pred["bbox"]
            max_iou = 0
            best_gt_idx = -1
            
            for gt_idx, gt in enumerate(gt_boxes):
                if gt_matched[gt_idx]:
                    continue  # Skip already matched ground truth boxes
                
                # Convert ground truth box to absolute coordinates
                gt_bbox_abs = convert_yolo_to_absolute(gt["bbox_normalized"], image_width, image_height)
                
                # Calculate IoU
                iou = calculate_iou(pred_bbox, gt_bbox_abs)
                
                if iou > max_iou:
                    max_iou = iou
                    best_gt_idx = gt_idx
            
            # If we found a match above the threshold
            if max_iou >= iou_threshold and best_gt_idx >= 0:
                gt_matched[best_gt_idx] = True
                pred_matched[pred_idx] = True
                tp += 1
                y_true.append(1)  # True positive
                y_pred.append(1)  # Predicted positive
            else:
                fp += 1
                y_true.append(0)  # Not a true positive
                y_pred.append(1)  # Predicted positive (false positive)
        
        # Count unmatched ground truth boxes as false negatives
        fn += sum(1 for matched in gt_matched if not matched)
        for gt_idx, matched in enumerate(gt_matched):
            if not matched:
                y_true.append(1)  # True positive that was missed
                y_pred.append(0)  # Predicted negative (false negative)
    
    print(f"True Positives: {tp}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    
    return y_true, y_pred

def plot_confusion_matrix(y_true, y_pred, output_file='confusion_matrix.png'):
    """Plot confusion matrix with true labels on x-axis and predictions on y-axis."""
    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Create labels for the plot
    class_names = ['Negative', 'Positive']
    
    # Create figure
    plt.figure(figsize=(8, 6))
    
    # Create heatmap with annotations
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    
    # Set labels (true on x-axis and pred on y-axis as requested)
    plt.ylabel('Predicted')
    plt.xlabel('True')
    plt.title('Confusion Matrix')
    
    # Save figure
    plt.savefig(output_file, bbox_inches='tight', dpi=300)
    plt.close()
    
    # Calculate metrics
    if len(cm) == 2 and len(cm[0]) == 2:  # Ensure it's a 2x2 matrix
        tn, fp, fn, tp = cm.ravel()
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        print(f"True Negatives: {tn}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1_score:.4f}")
    
    return cm

def get_image_dimensions(image_dir):
    """Get dimensions of the first image to use for normalization"""
    import cv2
    image_files = glob.glob(os.path.join(image_dir, "**/*.png"), recursive=True)
    if not image_files:
        return 1920, 1920  # Default if no images found
        
    # Read first image
    first_image = cv2.imread(image_files[0])
    if first_image is None:
        return 1920, 1920  # Default if image can't be read
        
    h, w = first_image.shape[:2]
    return w, h

def main():
    # File paths - update these to your actual paths
    predictions_file = "/mnt/storage/ji/YOLO8/runs/val/cmb_train_ALFA_resample_ESRGAN_t2s_3slices_cmbTrainOnly_conf0.01_0415_093907/csf_filter_predictions.json"
    labels_dir = "/mnt/storage/ji/brain_mri_valdo_mayo/valdo_resample_ALFA_YOLO_PNG_epd_gt_box_t2s_GAN_3slices_cmbTrainOnly/labels/val/"
    images_dir = "/mnt/storage/ji/brain_mri_valdo_mayo/valdo_resample_ALFA_YOLO_PNG_epd_gt_box_t2s_GAN_3slices_cmbTrainOnly/images/val/"
    
    # Get image dimensions for converting normalized boxes to absolute
    print("Getting image dimensions...")
    # image_width, image_height = get_image_dimensions(images_dir)
    image_width, image_height = 2048, 2048
    print(f"Using image dimensions: {image_width}x{image_height}")
    
    # Load predictions
    print("Loading predictions...")
    predictions = load_predictions(predictions_file)
    
    # Load ground truth
    print("Loading ground truth...")
    ground_truth = load_ground_truth(labels_dir)
    
    # Match predictions with ground truth and evaluate
    print("Evaluating predictions...")
    y_true, y_pred = evaluate_predictions(predictions, ground_truth, 
                                         image_width=image_width, 
                                         image_height=image_height)
    
    # Calculate and visualize confusion matrix
    print("Generating confusion matrix...")
    cm = plot_confusion_matrix(y_true, y_pred)
    
    print("Done!")
    return cm

if __name__ == "__main__":
    main()

Getting image dimensions...
Using image dimensions: 2048x2048
Loading predictions...
Loading ground truth...
Evaluating predictions...
True Positives: 77
False Positives: 1817
False Negatives: 49
Generating confusion matrix...
Confusion matrix saved as confusion_matrix.png
True Negatives: 0
False Positives: 1817
False Negatives: 49
True Positives: 77
Precision: 0.0407
Recall: 0.6111
F1 Score: 0.0762
Done!


In [80]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import glob
from collections import defaultdict
from sklearn.metrics import confusion_matrix
import seaborn as sns

def load_predictions(predictions_file):
    """Load YOLO predictions from JSON file."""
    with open(predictions_file, 'r') as f:
        predictions = json.load(f)
    
    # Group predictions by image_id
    pred_by_image = defaultdict(list)
    for pred in predictions:
        image_id = pred["image_id"]
        bbox = pred["bbox"]  # [x, y, width, height] format
        score = pred["score"]
        category_id = pred["category_id"]
        
        pred_by_image[image_id].append({
            "bbox": bbox,
            "score": score,
            "category_id": category_id
        })
    
    return pred_by_image

def load_ground_truth(labels_dir):
    """Load ground truth YOLO labels from directory."""
    gt_by_image = {}
    
    # Get all .txt files in the labels directory
    label_files = glob.glob(os.path.join(labels_dir, "**/*.txt"), recursive=True)
    
    for label_file in label_files:
        # Extract image_id from filename
        filename = os.path.basename(label_file)
        image_id = os.path.splitext(filename)[0]
        
        boxes = []
        # YOLO format: class_id x_center y_center width height (normalized)
        with open(label_file, 'r') as f:
            for line in f:
                if line.strip():
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        x_center = float(parts[1])
                        y_center = float(parts[2])
                        width = float(parts[3])
                        height = float(parts[4])
                        
                        boxes.append({
                            "class_id": class_id,
                            "bbox_normalized": [x_center, y_center, width, height]
                        })
        
        gt_by_image[image_id] = boxes
    
    return gt_by_image

def convert_yolo_to_absolute(bbox_normalized, image_width=1920, image_height=1920):
    """Convert YOLO normalized bbox to absolute coordinates [x, y, w, h]"""
    x_center, y_center, width, height = bbox_normalized
    
    x = (x_center - width/2) * image_width
    y = (y_center - height/2) * image_height
    w = width * image_width
    h = height * image_height
    
    return [x, y, w, h]

def calculate_iou(box1, box2):
    """
    Calculate IoU between two bounding boxes
    Both boxes in format [x, y, w, h] (absolute coordinates)
    """
    # Calculate coordinates of both boxes
    x1_1, y1_1 = box1[0], box1[1]
    x2_1, y2_1 = box1[0] + box1[2], box1[1] + box1[3]
    
    x1_2, y1_2 = box2[0], box2[1]
    x2_2, y2_2 = box2[0] + box2[2], box2[1] + box2[3]
    
    # Calculate intersection area
    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)
    
    if x2_i < x1_i or y2_i < y1_i:
        return 0.0  # No intersection
        
    intersection_area = (x2_i - x1_i) * (y2_i - y1_i)
    
    # Calculate union area
    box1_area = box1[2] * box1[3]
    box2_area = box2[2] * box2[3]
    union_area = box1_area + box2_area - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area if union_area > 0 else 0
    
    return iou

def evaluate_predictions(predictions, ground_truth, iou_threshold=0.5, conf_threshold=0.01, image_width=1920, image_height=1920):
    """
    Evaluate predictions against ground truth
    Returns true and predicted labels for confusion matrix
    """
    y_true = []  # Ground truth labels 
    y_pred = []  # Predicted labels
    
    # Track metrics
    tp = 0  # True positives
    fp = 0  # False positives
    fn = 0  # False negatives
    
    for image_id, gt_boxes in ground_truth.items():
        # Get predictions for this image
        image_preds = predictions.get(image_id, [])
        
        # Filter by confidence threshold
        image_preds = [p for p in image_preds if p["score"] >= conf_threshold]
        
        # Initialize tracking arrays for this image
        gt_matched = [False] * len(gt_boxes)
        pred_matched = [False] * len(image_preds)
        
        # For each prediction, find best matching ground truth box
        for pred_idx, pred in enumerate(image_preds):
            pred_bbox = pred["bbox"]
            max_iou = 0
            best_gt_idx = -1
            
            for gt_idx, gt in enumerate(gt_boxes):
                if gt_matched[gt_idx]:
                    continue  # Skip already matched ground truth boxes
                
                # Convert ground truth box to absolute coordinates
                gt_bbox_abs = convert_yolo_to_absolute(gt["bbox_normalized"], image_width, image_height)
                
                # Calculate IoU
                iou = calculate_iou(pred_bbox, gt_bbox_abs)
                
                if iou > max_iou:
                    max_iou = iou
                    best_gt_idx = gt_idx
            
            # If we found a match above the threshold
            if max_iou >= iou_threshold and best_gt_idx >= 0:
                gt_matched[best_gt_idx] = True
                pred_matched[pred_idx] = True
                tp += 1
                y_true.append(1)  # True positive
                y_pred.append(1)  # Predicted positive
            else:
                fp += 1
                y_true.append(0)  # Not a true positive
                y_pred.append(1)  # Predicted positive (false positive)
        
        # Count unmatched ground truth boxes as false negatives
        fn += sum(1 for matched in gt_matched if not matched)
        for gt_idx, matched in enumerate(gt_matched):
            if not matched:
                y_true.append(1)  # True positive that was missed
                y_pred.append(0)  # Predicted negative (false negative)
    
    print(f"True Positives: {tp}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    
    return y_true, y_pred

def plot_confusion_matrix(y_true, y_pred, output_file='confusion_matrix.png'):
    """Plot confusion matrix with true labels on x-axis and predictions on y-axis."""
    # Generate confusion matrix from sklearn
    cm_original = confusion_matrix(y_true, y_pred)
    
    # We need to manually create the confusion matrix in the desired format
    # In your image format:
    # [cmb_pred_as_cmb, background_pred_as_cmb]
    # [cmb_pred_as_background, background_pred_as_background]
    
    if len(cm_original) == 2 and len(cm_original[0]) == 2:
        tn, fp, fn, tp = cm_original.ravel()
        
        # Reorder the matrix to match your desired format
        # The new matrix will have:
        # [TP, FP]  # True cmb detected as cmb, Background detected as cmb
        # [FN, TN]  # True cmb detected as background, Background detected as background
        cm = np.array([
            [tp, fp],
            [fn, tn]
        ])
    else:
        # If for some reason the matrix is not 2x2, use a default format
        cm = cm_original
    
    # Create labels for the plot
    class_names = ['cmb', 'background']
    
    # Create figure with border
    plt.figure(figsize=(12, 10), facecolor='white', edgecolor='black', linewidth=2)
    
    # Create heatmap with annotations
    cmap = sns.color_palette("Blues", as_cmap=True)
    ax = sns.heatmap(cm, annot=True, fmt='d', cmap=cmap,
                xticklabels=class_names, yticklabels=class_names,
                annot_kws={"size": 30, "weight": "bold"},
                cbar_kws={"shrink": .8})
    
    # Increase font size of tick labels
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=12)
    
    # Set labels and title
    plt.ylabel('Predicted', fontsize=14)
    plt.xlabel('True', fontsize=14)
    plt.title('Confusion Matrix', fontsize=16)
    
    # Save figure with high resolution
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Output metrics
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"Confusion matrix saved as {output_file}")
    print(f"True Positives (cmb correctly identified as cmb): {tp}")
    print(f"False Positives (background incorrectly identified as cmb): {fp}")
    print(f"False Negatives (cmb incorrectly identified as background): {fn}")
    print(f"True Negatives (background correctly identified as background): {tn}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    
    return cm

def get_image_dimensions(image_dir):
    """Get dimensions of the first image to use for normalization"""
    import cv2
    image_files = glob.glob(os.path.join(image_dir, "**/*.png"), recursive=True)
    if not image_files:
        return 1920, 1920  # Default if no images found
        
    # Read first image
    first_image = cv2.imread(image_files[0])
    if first_image is None:
        return 1920, 1920  # Default if image can't be read
        
    h, w = first_image.shape[:2]
    return w, h

def main():
    # File paths - update these to your actual paths
    predictions_file = "/mnt/storage/ji/YOLO8/runs/val/cmb_train_ALFA_resample_ESRGAN_t2s_3slices_cmbTrainOnly_conf0.01_0415_093907/csf_filter_predictions.json"
    labels_dir = "/mnt/storage/ji/brain_mri_valdo_mayo/valdo_resample_ALFA_YOLO_PNG_epd_gt_box_t2s_GAN_3slices_cmbTrainOnly/labels/val/"
    images_dir = "/mnt/storage/ji/brain_mri_valdo_mayo/valdo_resample_ALFA_YOLO_PNG_epd_gt_box_t2s_GAN_3slices_cmbTrainOnly/images/val/"
    
    # Get image dimensions for converting normalized boxes to absolute
    print("Getting image dimensions...")
    # image_width, image_height = get_image_dimensions(images_dir)
    image_width, image_height = 2048, 2048
    print(f"Using image dimensions: {image_width}x{image_height}")
    
    # Load predictions
    print("Loading predictions...")
    predictions = load_predictions(predictions_file)
    
    # Load ground truth
    print("Loading ground truth...")
    ground_truth = load_ground_truth(labels_dir)
    
    # Match predictions with ground truth and evaluate
    print("Evaluating predictions...")
    y_true, y_pred = evaluate_predictions(predictions, ground_truth, 
                                         image_width=image_width, 
                                         image_height=image_height)
    
    # Calculate and visualize confusion matrix
    print("Generating confusion matrix...")
    cm = plot_confusion_matrix(y_true, y_pred, output_file='confusion_matrix.png')
    
    print("Done!")
    return cm

if __name__ == "__main__":
    main()

Getting image dimensions...
Using image dimensions: 2048x2048
Loading predictions...
Loading ground truth...
Evaluating predictions...
True Positives: 77
False Positives: 1817
False Negatives: 49
Generating confusion matrix...
Confusion matrix saved as confusion_matrix.png
True Positives (cmb correctly identified as cmb): 77
False Positives (background incorrectly identified as cmb): 1817
False Negatives (cmb incorrectly identified as background): 49
True Negatives (background correctly identified as background): 0
Precision: 0.0407
Recall: 0.6111
F1 Score: 0.0762
Done!
